In [1]:
import jax
import jax.numpy as jnp
from flax.struct import dataclass

In [2]:
@dataclass
class Gaussian:
    info: jnp.ndarray
    precision: jnp.ndarray
    dims: jnp.ndarray 

    @property
    def shape(self):
        return {
            "info": self.info.shape,
            "precision": self.precision.shape,
            "dims": self.dims.shape
        }
 
    @property
    def mean(self) -> jnp.ndarray:
        return jnp.linalg.inv(self.precision) @ self.info
    
    @property
    def covariance(self) -> jnp.ndarray:
        return jnp.linalg.inv(self.precision)
    
    @staticmethod
    def identity(variable: int) -> jnp.ndarray:
        dims = jnp.array([variable, variable, variable, variable])
        return Gaussian(jnp.zeros(4), jnp.eye(4), dims)
    
    def concatenate(self, other_gaussian: "Gaussian") -> "Gaussian":
        return Gaussian(
            jnp.concatenate(self.info, other_gaussian.info),
            jnp.concatenate(self.precision, other_gaussian.precision),
            jnp.concatenate(self.dims, other_gaussian.dims)
        )
    
    def __getitem__(self, index) -> "Gaussian":
        return Gaussian(self.info[index], self.precision[index], self.dims[index])

    def __mul__(self, other: 'Gaussian') -> 'Gaussian':
        if other is None:
            return self.copy()

        _info, _prec = self.info, self.precision        
        # Merge dims
        dims = list(self.dims)
        for d in other.dims:
            if d not in dims:
                dims.append(d)
        print("dims:", dims)
        # Extend self matrix
        prec_self = jnp.zeros((len(dims), len(dims)))
        info_self = jnp.zeros((len(dims), 1))
        idxs_self = jnp.array([dims.index(d) for d in self.dims])
        jax.debug.print("idxs self: {}", idxs_self)
        prec_self = prec_self.at[jnp.ix_(idxs_self, idxs_self)].set(_prec)
        info_self = info_self.at[jnp.ix_(idxs_self, jnp.array([0]))].set(_info.reshape(-1,1))
        # Extend other matrix
        prec_other = jnp.zeros((len(dims), len(dims)))
        info_other = jnp.zeros((len(dims), 1))
        idxs_other = jnp.array([dims.index(d) for d in other.dims])
        jax.debug.print("idxs other: {}", idxs_other)
        prec_other = prec_other.at[jnp.ix_(idxs_other, idxs_other)].set(other.precision)
        info_other = info_other.at[jnp.ix_(idxs_other, jnp.array([0]))].set(other.info.reshape(-1,1))
        # Add
        prec = prec_other + prec_self
        info = info_other + info_self
        return Gaussian(info.squeeze(), prec, dims)

    def __imul__(self, other: 'Gaussian') -> 'Gaussian':
        return self.__mul__(other)

    def marginalize(self, dims: list):
        """Given dims will be marginalized out.
        """
        info, prec = self.info, self.precision
        info = info.reshape(-1,1)
        axis_a = jnp.array([idx for idx, d in enumerate(self.dims) if d not in dims])
        axis_b = jnp.array([idx for idx, d in enumerate(self.dims) if d in dims])

        jax.debug.print("axis a: {}", axis_a)
        jax.debug.print("axis b: {}", axis_b)

        info_a = info[jnp.ix_(axis_a, jnp.array([0]))]
        prec_aa = prec[jnp.ix_(axis_a, jnp.array(axis_a))]
        info_b = info[jnp.ix_(axis_b, jnp.array([0]))]
        prec_ab = prec[jnp.ix_(axis_a, axis_b)]
        prec_ba = prec[jnp.ix_(axis_b, axis_a)]
        prec_bb = prec[jnp.ix_(axis_b, axis_b)]

        prec_bb_inv = jnp.linalg.inv(prec_bb)
        info_ = info_a - prec_ab @ prec_bb_inv @ info_b
        prec_ = prec_aa - prec_ab @ prec_bb_inv @ prec_ba

        new_dims = tuple(d for d in self.dims if d not in dims)
        return Gaussian(info_.squeeze(), prec_, new_dims)

In [3]:
same_g0 = Gaussian(jnp.ones(4,) * 1, jnp.eye(4) * 1, ['x1.x', 'x1.y', 'x1.vx', 'x1.vy'])
same_g1 = Gaussian(jnp.ones(4,) * 1, jnp.eye(4) * 1, ['x1.x', 'x1.y', 'x1.vx', 'x1.vy'])
same_g0 * same_g1

dims: ['x1.x', 'x1.y', 'x1.vx', 'x1.vy']
idxs self: [0 1 2 3]
idxs other: [0 1 2 3]


Gaussian(info=Array([2., 2., 2., 2.], dtype=float32), precision=Array([[2., 0., 0., 0.],
       [0., 2., 0., 0.],
       [0., 0., 2., 0.],
       [0., 0., 0., 2.]], dtype=float32), dims=['x1.x', 'x1.y', 'x1.vx', 'x1.vy'])

In [4]:
g0 = Gaussian(jnp.ones(4,) * 1, jnp.eye(4) * 1, ['x1.x', 'x1.y', 'x1.vx', 'x1.vy'])
g1 = Gaussian(jnp.ones(4,) * 2, jnp.eye(4) * 2, ['x2.x', 'x2.y', 'x2.vx', 'x2.vy'])
g2 = Gaussian(jnp.ones(4,) * 3, jnp.eye(4) * 3, ['x3.x', 'x3.y', 'x3.vx', 'x3.vy'])
combined = g0 * g1 * g2
combined

dims: ['x1.x', 'x1.y', 'x1.vx', 'x1.vy', 'x2.x', 'x2.y', 'x2.vx', 'x2.vy']
idxs self: [0 1 2 3]
idxs other: [4 5 6 7]
dims: ['x1.x', 'x1.y', 'x1.vx', 'x1.vy', 'x2.x', 'x2.y', 'x2.vx', 'x2.vy', 'x3.x', 'x3.y', 'x3.vx', 'x3.vy']
idxs self: [0 1 2 3 4 5 6 7]
idxs other: [ 8  9 10 11]


Gaussian(info=Array([1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.], dtype=float32), precision=Array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3.]], dtype=float32), dims=['x1.x', 'x1.y', 'x1.vx', 'x1.vy', 'x2.x', 'x2.y', 'x2.vx', 'x2.vy', 'x3.x', 'x3.y', 'x3.vx', 'x3.vy'])

In [5]:
combined * Gaussian(jnp.ones(4,) * 1, jnp.eye(4) * 1, ['x1.x', 'x1.y', 'x1.vx', 'x1.vy'])

dims: ['x1.x', 'x1.y', 'x1.vx', 'x1.vy', 'x2.x', 'x2.y', 'x2.vx', 'x2.vy', 'x3.x', 'x3.y', 'x3.vx', 'x3.vy']
idxs self: [ 0  1  2  3  4  5  6  7  8  9 10 11]
idxs other: [0 1 2 3]


Gaussian(info=Array([2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3.], dtype=float32), precision=Array([[2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3.]], dtype=float32), dims=['x1.x', 'x1.y', 'x1.vx', 'x1.vy', 'x2.x', 'x2.y', 'x2.vx', 'x2.vy', 'x3.x', 'x3.y', 'x3.vx', 'x3.vy'])

In [6]:
combined.marginalize(['x2.x', 'x2.y', 'x2.vx', 'x2.vy'])

axis a: [ 0  1  2  3  8  9 10 11]
axis b: [4 5 6 7]


Gaussian(info=Array([1., 1., 1., 1., 3., 3., 3., 3.], dtype=float32), precision=Array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 3., 0., 0., 0.],
       [0., 0., 0., 0., 0., 3., 0., 0.],
       [0., 0., 0., 0., 0., 0., 3., 0.],
       [0., 0., 0., 0., 0., 0., 0., 3.]], dtype=float32), dims=('x1.x', 'x1.y', 'x1.vx', 'x1.vy', 'x3.x', 'x3.y', 'x3.vx', 'x3.vy'))